Homework Sheet — Stage 04: Data Acquisition and Ingestion

In [1]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


PART 1: API PULL

In [2]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

In [6]:
SYMBOL = 'IBM'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))

if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {
        'function':'TIME_SERIES_DAILY',
        'symbol':SYMBOL,
        'outputsize':'compact',
        'apikey': os.getenv('ALPHAVANTAGE_API_KEY')
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    print(js)
    key = [k for k in js if 'Time Series' in k][0]
    
    # Use '4. close' instead of adjusted close
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(
        columns={'index':'date','4. close':'close'}
    )[['date','close']]
    df_api['date'] = pd.to_datetime(df_api['date'])
    df_api['close'] = pd.to_numeric(df_api['close'])

else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Close']]
    df_api.columns = ['date','close']

# Validate
v_api = validate(df_api, ['date','close'])
v_api

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'IBM', '3. Last Refreshed': '2025-08-20', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2025-08-20': {'1. open': '242.1100', '2. high': '242.8800', '3. low': '240.3400', '4. close': '242.5500', '5. volume': '3209751'}, '2025-08-19': {'1. open': '240.0000', '2. high': '242.8300', '3. low': '239.4900', '4. close': '241.2800', '5. volume': '3328305'}, '2025-08-18': {'1. open': '239.5700', '2. high': '241.4200', '3. low': '239.1158', '4. close': '239.4500', '5. volume': '3569594'}, '2025-08-15': {'1. open': '237.6100', '2. high': '240.6200', '3. low': '236.7700', '4. close': '239.7200', '5. volume': '4344322'}, '2025-08-14': {'1. open': '238.2500', '2. high': '239.0000', '3. low': '235.6200', '4. close': '237.1100', '5. volume': '4556725'}, '2025-08-13': {'1. open': '236.2000', '2. high': '240.8411', '3. low': '236.2000', '4. close': '240.0700', '5. volu

{'missing': [], 'shape': (100, 2), 'na_total': 0}

In [ ]:
SAVING THE API RAW CSV TO data/raw

In [7]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)
print(_)

Saved data\raw\api_source-alpha_symbol-IBM_20250821-041332.csv
data\raw\api_source-alpha_symbol-IBM_20250821-041332.csv


SCRAPING FROM WEB USING BEAUTIFULSOUP

In [14]:
stock_ticker = "GOOGL:NASDAQ"
google_finance_quote_url = f"https://www.google.com/finance/quote/{stock_ticker}"
print(google_finance_quote_url)

headers = {'User-Agent':'AFE-Homework/1.0'}
try:
    resp = requests.get(google_finance_quote_url, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)
    print(df_scrape)
except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

https://www.google.com/finance/quote/GOOGL:NASDAQ
                                                (USD)  \
0   RevenueThe total amount of income generated by...   
1   Operating expenseRepresents the total incurred...   
2   Net incomeCompany’s earnings for a period net ...   
3   Net profit marginMeasures how much net income ...   
4   Earnings per shareRepresents the company's pro...   
5   EBITDAEarnings before interest, taxes, depreci...   
6   Effective tax rateThe percent of their income ...   
7                                               (USD)   
8   Cash and short-term investmentsInvestments tha...   
9   Total assetsThe total amount of assets owned b...   
10  Total liabilitiesSum of the combined debts a c...   
11  Total equityThe value of subtracting the total...   
12  Shares outstandingTotal number of common share...   
13  Price to bookA ratio used to determine if a co...   
14  Return on assetsA financial ratio that shows a...   
15  Return on capitalCompany’s return 

{'missing': [], 'shape': (23, 3), 'na_total': 0}

SAVING THE SCRAPE RAW CSV TO data/raw

In [15]:
_ = save_csv(df_scrape, prefix='scrape', site='google_finance', stock_ticker='GOOGL:NASDAQ')
print(_)

Saved data\raw\scrape_site-google_finance_stock_ticker-GOOGL:NASDAQ_20250821-041759.csv
data\raw\scrape_site-google_finance_stock_ticker-GOOGL:NASDAQ_20250821-041759.csv
